<font size="6">Рекуррентные нейронные сети</font>

# Обработка последовательных данных

В предыдущей лекции мы научились строить векторные представления для каждого слова в предложении. При этом векторное представление всего текста представляло собой усреднение векторов для каждого слова.

У этого подхода есть важный минус: он не учитывает, что для анализа текстовых данных важна их последовательность. Тем самым, тексты **«Я не люблю ML»** и **«Я люблю не ML»** получают **одинаковые** векторизации, то есть по ходу **теряется** существенная информация.

Текст пишется и читается **последовательно**. Нам может показаться, что это стоит использовать: при обработке очередного токена обращаться к предыдущим, как к его **контексту**.

Для обработки последовательных данных могут использоваться **рекуррентные нейронные сети (recurrent neural networks, RNN)**. Они применяются в широком перечне задач: от **распознавания речи** до **генерации подписей** к изображениям.

Эти задачи объединяет необходимость работы с **контекстом** и извлечения информации, сформированной при обработке одной части данных, для обработки других частей.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/sequence_data.png" width="1000"></center>

С моделями рекуррентных нейронных сетей конкурируют модели, основанные на архитектуре **Трансформер** (подробнее в следующей лекции).

Хотя сейчас трансформеры держат первенство во многих областях, для их обучения в силу их размера требуется:
- несоизмеримо большее количество данных, нежели для RNN;
- больше вычислительных ресурсов как для обучения, так и для инференса.

# Архитектура рекуррентных нейронных сетей

### Рекуррентный слой в нейронных сетях

Рассмотрим работу рекуррентной нейронной сети при обработке текстовых данных:
1. На вход поступает последовательность слов $\large x = \{x_1,...x_t,...,x_T\}$, где $\large x_t$ — вектор слова с индексом $t$.

2. Для каждого поступившего $\large x_t$ формируем скрытое состояние $\large h_t$, которое фактически является линейным преобразованием от предыдущего состояния $\large h_{t-1}$ и текущего элемента последовательности $\large x_t$, к которому применяется нелинейная функция активации:
$$\large h_t = f_{\text{act}}(W_{hh}h_{t-1} + W_{xh}x_t),$$
где $\large W_{hh}$ и $\large W_{xh}$  — это матрицы обучаемых параметров (веса). Также может добавляться вектор смещений (bias).

Когда первый токен $\large x_0$ подается в ячейку, скрытое состояние $\large h_0$ инициализируется нулями.

3. На основании рассчитанного скрытого состояния, учитывающего предыдущие значения  $\large x_t$, формируется выходная последовательность $\large y = \{y_1,...y_t,...,y_T\}$. Для формирования предсказания $\large y_t$ в текущий момент времени в модель добавляется обычный линейный слой, принимающий на вход текущее скрытое состояние $\large h_t$.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/rnn_basic_block.png" width="1000"></center>


Таким образом, для каждого последующего токена скрытое состояние учитывает контекст из предыдущих токенов.

Вектор $\large y_t$ является не только контекстуализированным представлением для последнего токена $\large x_t$, но и вектором всего предложения, т.к. содержит информацию обо всех токенах. В зависимости от типа задачи, можно использовать контекстуализированные вектора для каждого слова или только вектор всего предложения (последнего слова).

## RNN слой в PyTorch

В PyTorch есть слой — `torch.nn.RNN` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html), который реализует логику, описанную выше.

Также есть сущность `torch.nn.RNNCell` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.RNNCell.html), которая реализует вычисления на одном такте времени.

Слой `nn.RNN` фактически является оберткой, которая вызывает `nn.RNNCell` в цикле по длине входной последовательности.

Параметры слоя `nn.RNN`:

* **`input_size`** — размерность $\large x_t$, целое число.

* **`hidden_size`** — размерность $\large h_t$, целое число. Фактически это количество нейронов в рекуррентном слое.


Техническая особенность рекуррентных слоев в PyTorch: по умолчанию ожидаемые размерности входа такие:

**`[длина последовательности, размер батча, размерность входа]`**

Однако если при создании слоя указать `batch_first=True`, то можно подавать входные значения в более привычном формате, когда размер батча стоит на первом месте:

**`[размер батча, длина последовательности, размерность входа]`**



In [ ]:
import torch

torch.manual_seed(42)

rnn = torch.nn.RNN(input_size=3, hidden_size=2, batch_first=True)

dummy_batched_seq = torch.randn((16, 57, 3))  # batch_size, seq_len, input_size
out, h = rnn(dummy_batched_seq)

print("Input shape:".ljust(20), f"{dummy_batched_seq.shape}")
print("Out shape:".ljust(20), f"{out.shape}")
print("Last hidden state shape:".ljust(20), f"{h.shape}")

При вызове слой возвращает два объекта:
* `out` — последовательность скрытых состояний,
* `h` — скрытое состояние на последнем такте.

Мы указали `batch_first=True`, при этом `out` сохранил последовательность размерностей, как у входа, а вот у `h` размерность батча встала на второе место.

In [ ]:
h_batch_first = h.permute(1, 0, 2)

print(f"h is last out: {(h_batch_first == out[:, -1:, :]).all().item()}")

### Многослойные RNN

RNN-блоки можно объединять в слои, накладывая их друг на друга. Для этой операции в `torch.nn.RNN` есть аргумент `num_layers`, с помощью которого можно указать количество слоёв.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/rnn_multiple_layers.png" width="500"></center>

In [ ]:
dummy_input = torch.randn((16, 6, 3))  # batch_size, seq_len, input_size
rnn = torch.nn.RNN(input_size=3, hidden_size=2, num_layers=2, batch_first=True)

out, h = rnn(dummy_input)

print()
print("Out:\n", out.shape)  # Hidden states for all elements from top layer
print("h:\n", h.shape)  # Hidden states for last element for all layers

### Bidirectional RNN

В текстах начало или окончание предложения могут полностью изменить его смысл. Так, в задачах оценки эмоциональной окраски, можно легко ошибиться, если обрабатывать предложение только в одном направлении, постепенно забывая начало.

Двунаправленная зависимость является важнейшей особенностью, которую мы будем учитывать.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/movie_sentiment.png" width="650"></center>

Последовательность можно пропустить через рекуррентную сеть два раза: при просмотре слов в прямом и в обратном направлении. Для этого создаётся еще один рекуррентный слой, который обрабатывает последовательность в обратном направлении, а скрытые состояния двух слоёв конкатенируется.

[[blog] ✏️ Recurrent Neural Networks with PyTorch](https://www.kaggle.com/code/kanncaa1/recurrent-neural-network-with-pytorch)

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/bidirectional.png" width="650"></center>

За реализацию такой двунаправленности в рекуррентных слоях отвечает флаг `bidirectional=True`.

In [ ]:
dummy_input = torch.randn((16, 57, 3))  # batch_size, seq_len, input_size
rnn = torch.nn.RNN(3, 2, bidirectional=True, batch_first=True)

out, h = rnn(dummy_input)

# Concatenated Hidden states from both layers
print("Out:\n", out.shape)
# Hidden states last element from  both : 2*num_layers*hidden_state
print("h:\n", h.shape)

## Проблемы RNN

Теоретически, можно было бы сразу пропустить все данные через сеть и затем вычислить градиент, однако возникнут следующие проблемы:

 - большие последовательности не поместятся в памяти,
 - так как цепочка будет очень длинной, возникнет затухание/взрыв градиента,
 - по мере прохождения сигнала по цепи контекст затирается.


Допустим, у нас есть длинная последовательность. Если мы сразу предсказываем, то в каждый момент времени нужно распространить Loss. И все ячейки нужно обновить во время backpropogation. Все градиенты нужно посчитать. Возникают проблемы, связанные с нехваткой памяти.

Есть специальные тесты для проверки, контекст какой длины использует RNN при предсказании. Если мы делаем предсказание только в последней ячейке, может оказаться, что используется, скажем, информация только о последних 10 словах предложения.

Функция активации Tanh постепенно затирает контекст.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/backprop_through_time.png" width="700"><center>

<center><em>Источник: <a href="http://cs231n.stanford.edu/slides/2021/lecture_10.pdf">CS231n: Recurrent Neural Network</a></em></center>

Затухающий/взрывающийся градиент (Vanishing/exploding gradient) — явления затухающего и взрывающегося градиента часто встречаются в контексте RNN. И при большой длине последовательности это становится критичным. Причина в том, что зависимость величины градиента от числа слоёв экспоненциальная, поскольку веса умножаются многократно.

$dL ∝ (W)^N:$

$W > 1 \rightarrow$ взрыв, $W < 1 \rightarrow$ затухание.

<img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/simple_rnn_backprop.png" width="1000">

## LSTM (Long Short-Term Memory)

Обычная RNN имела множество проблем, в том числе в ней очень быстро затухала информация о предыдущих элементах последовательности. Помимо этого были проблемы с затуханием/взрывом градиента.

Эти проблемы были частично решены в LSTM, предложенной в статье:

[[article]🎓Hochreiter S., Schmidhuber J. (1997). Long Short-Term Memory](http://www.bioinf.jku.at/publications/older/2604.pdf).

В обычной RNN-ячейке был только один путь передачи информации. На каждом шаге мы конкатенировали предыдущее скрытое состояние с текущим входом и пропускали их через линейный слой и активацией:

<div align="center">
<html>
<head>
<style>
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
<font size="2" face="Times New Romans" >


</style>
</head>
<body>

<table >

<tr>
<td>

<center><img src = "https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/simple_rnn_h_state.png" width="500"></center>

</td>
<td>

<table >
<tr>
<td>

$\large h_t = \tanh(W \cdot [h_{t-1}, x_t])$

</td>

</tr>


</table>

</td>
</tr>

</table>








При этом информация о предыдущих элементах последовательности очень быстро затухает и теряется общая информация о предложении.

Структура ячейки LSTM намного сложнее. Здесь есть целых 4 линейных слоя, каждый из которых выполняет разные задачи.

<div align="center">
<html>
<head>
<style>
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
<font size="2" face="Times New Romans" >


</style>
</head>
<body>

<table >
<tr>
<td>

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/lstm_chain.png" width="500"></center>
<center><img src="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/lstm_chain_notation.png" width="700"></center>

</td>
<td>

<table >
<tr>
<td>

$\large f_t = σ(W_f \cdot [h_{t-1}, x_t])\ \ \ \ $

</td>
<td>

$$\large \text{forget  gate}$$

</td>
</tr>

<tr>
<td>

$\large i_t = σ(W_i \cdot [h_{t-1}, x_t])$

</td>
<td>

$$\large \text{input gate}$$

</td>
</tr>

<tr>
<td>

$\large o_t = σ(W_o \cdot [h_{t-1}, x_t])$

</td>
<td>

$$\large \text{output gate}$$

</td>
</tr>

<tr>
<td>

$\large c^\prime_t = \tanh(W_c \cdot [h_{t-1}, x_t])$

</td>
<td>

$$\large \text{candidate cell state}$$

</td>
</tr>

<tr>
<td>

$\large c_t = f_t\otimes c_{t-1} + i_t \otimes c^\prime_t$

</td>
<td>

$$\large \text{cell state}$$

</td>
</tr>

<tr>
<td>

$\large h_t = o_t\otimes \tanh(c_t)$

</td>
<td>

$$\large  \text{hidden state}$$

</td>
</tr>
</table>

</td>
</tr>

</table>








Главное нововведение: в LSTM добавлен путь $c$, который по задумке должен этот общий контекст сохранять.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/lstm_c_state_highway.png" width="500"></center>

Другими словами, путь $c$ (cell state, иногда называется highway, магистраль)  помогает нейросети сохранять важную информацию, встретившуюся в какой-то момент в последовательности в прошлом, все время, пока эта информация требуется.

По формулам также видно, как возросла сложность.

Отличие от RNN состоит в том, что кроме $h$ возвращается еще и $c$.

In [ ]:
import torch.nn as nn


lstm = nn.LSTM(input_size=3, hidden_size=2, batch_first=True)
input = torch.randn(16, 57, 3)  # batch_size, seq_len, input_size
out, (h, c) = lstm(input)  # h and c returned in tuple

print("Input shape:".ljust(15), input.shape)
print("Shape of h".ljust(15), h.shape)  # 1, batch_size, hidden_size
print("Shape of c".ljust(15), c.shape)  # 1, batch_size, hidden_size
print("Output shape:".ljust(15), out.shape)  # batch_size, seq_len, hidden_size

## GRU (Gated Recurrent Unit)

Самая известная модификация LSTM — GRU. Она более компактна за счет сильных упрощений в сравнении со стандартной LSTM.

Главные изменения: объединены forget и input gates, слиты $h_t$ и $c_t$, которые в обычной LSTM только участвовали в формировании друг друга.

<div align="center">
<html>
<head>
<style>
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
<font size="2" face="Times New Romans" >


</style>
</head>
<body>

<table >
<tr>
<td>

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/gru_basic_block.png" width="500"></center>

</td>
<td>

<table >
<tr>
<td>

$\large z_t = \sigma(W_z \cdot [h_{t-1}, x_t])$

</td>

</tr>

<tr>
<td>

$\large r_t = \sigma(W_r \cdot [h_{t-1}, x_t])$

</td>

</tr>

<tr>
<td>

$\large \tilde h_t = \tanh(W \cdot [r_t \otimes h_{t-1}, x_t])$

</td>

</tr>

<tr>
<td>

$\large h_t = (1-z_t) \otimes h_{t-1} + z_t \otimes \tilde h_t$

</td>


</table>








</td>
</tr>

</table>








In [ ]:
gru = nn.GRU(input_size=3, hidden_size=2, batch_first=True)
input = torch.randn(16, 57, 3)  # batch_size, seq_len, input_size
out, h = gru(input)

print("Input shape:".ljust(15), input.shape)
print("Shape of h:".ljust(15), h.shape)  # 1, batch_size, hidden_size
print("Output shape:".ljust(15), out.shape)  # batch_size, seq_len, hidden_size

Практический опыт исследователей: иногда лучше работает GRU, иногда — LSTM. Точный рецепт успеха сказать нельзя.

## Типы задач

В зависимости от количества объектов на входе и количества ответов на выходе постановка задачи может быть разной.

Простейший вариант — **«оne to one»** (один объект на входе, один ответ на выходе). Например, **классификация изображений**: на вход подается один элемент, на выходе получаем вероятность класса. Для этой задачи подойдет **обычная нейронная сеть**, не обязательно применять RNN в таком случае.

Рассмотрим случаи, где оправдано использование **рекуррентной сети** оправдано:

1. Более сложная реализация — **«one to many»** (один объект на входе, необходимо сформировать несколько выходов). Такой тип нейронной сети актуален, когда мы говорим о **генерации**: например, текстов или музыки.
- задаем начальное слово или начальный звук;
- модель начинает самостоятельно генерировать выходы;
- вход к очередной ячейке — это выход с прошлой ячейки нейронной сети.
2. Задача **many-to-one** (несколько объектов на входе, один ответ на выходе). Рекуррентная сеть выдает ответ на каждом шаге, однако в этом случае используется только ответ, выданный на последнем шаге. Это релевантно для **классификации последовательных данных**: мы должны проанализировать все входы нейронной сети и только в конце определиться с классом.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/one_or_many_to_one_or_many_ways_1.png" width="700"></center>

Возможна постановка задачи **«many to many»** (несколько объектов на входе, несколько ответов на выходе) возможна в двух вариантах:

1. Количество **выходов** нейронной сети **может быть не равно** количеству **входов**. Это актуально в **машинном переводе**, когда одна и та же фраза может иметь **разное количество слов** в разных языках.

2. Количество **выходов** нейронной сети **равно** количеству **входов**. Обычно это задачи **разметки** исходной последовательности. Например, указать столицы городов, названия важных объектов, веществ и т.д., что относится к задачам вида NER (Named entity recogition).

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/one_or_many_to_one_or_many_ways_2.png" width="700"></center>

# RNN для языкового моделирования

Языковые модели — важнейшая часть современного NLP. Практически во всех задачах, связанных с обработкой текста, напрямую или косвенно используются языковые модели. А наиболее известные недавние прорывы в области — это по большей части новые подходы к языковому моделированию. ELMO, BERT, GPT — это языковые модели.

## Задача языкового моделирования

Языковая модель оценивает вероятность встретить предложение $S$  — последовательность слов $(w_1,\cdots ,w_n)$. Вероятность предложения можно определить как произведение вероятности каждого слова с учетом предыдущих слов:
$$P(w_1,w_2, \dots, w_n) = p(w_1)p(w_2|w_1)p(w_3|w_1,w_2)\dots p(w_n|w_1,w_2,\dots,w_{n-1})= \prod\limits_{i = 1}^n p(w_i|w_1, \dots, w_{i-1})$$
Для каждого слова последовательности предсказывается вероятность встретить его в тексте при условии, что известно предыдущее слово: $w_2$ при условии $w_1$, $w_3$ при условии $w_1$ и $w_2$, и т.д.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/sentence.png" width="700"><center>

<center><em>Источник: <a href="https://thegradient.pub/understanding-evaluation-metrics-for-language-models/">Evaluation Metrics for Language Modeling</a></em></center>

Истинные вероятности предложений неизвестны → можно обучить языковую модель оценивать эти вероятности. Вероятность рассчитывается на основе частоты встречаемости слов в корпусе текстов:
$$p (w_i|w_{i-(n-1)}, \dots, w_{i-1}) = \frac{count(w_{i-(n-1)}, \dots, w_{i-1}, w_{i})}{count(w_{i-(n-1)} \dots, w_{i-1})}$$
Такие языковые модели называются *n-граммными*. Термины *биграммные* и *триграммные* языковые модели обозначают n-граммные модели с $n = 2$ и $n = 3$ соответственно.

Результат: языковая модель способна оценивать вероятность следующего слова последовательности.

Вы наверняка сталкивались  использованием языковых моделей в повседневной жизни, когда при вводе сообщения или поискового запроса вам предлагались варианты его наиболее вероятного продолжения.

<div align="center">
    <table >
     <tr>
       <td>
       
<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/keyboard.png" width="400"><center>

<center><em>Источник: <a href="https://habr.com/ru/companies/ods/articles/716918/">Эволюция языковых моделей с T9 до чуда</a></em></center>

</td>

<td>

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/search.png" width="700"><center>
<em>Источник: поисковый запрос в Google</em>


</td>
     </tr>
    </table>
    </div>

Если модель способна предсказать вероятность следующего слова, то она уже достаточно много знает о языке.

*Я люблю вкусную ...*

На месте пропуска должно стоять неодушевленное существительное женского рода в винительном падеже, которое обозначает нечно съедбное (*еду, колбасу, рыбу* и т.д.).

Языковая модель может применяться для генерации текста: если подать на вход некоторое слово $w_1$, то модель предскажет наиболее вероятное продолжение $w_2$. Затем слова $w_1$ и $w_2$ используются для предсказания следующего слова $w_3$.

Основной недостаток счетных (*n*-граммных) языковых моделей в том, что они не способны учитывать длинный контекст предложения и генерировать связный текст.

С этими проблемами справляются языковые модели на основе рекуррентных нейронных сетей.

## Применение рекуррентных сетей

Рекуррентные нейронные сети должны хорошо подходить для языкового моделирования, поскольку они проходят по последовательности и запоминают порядок элементов. Рассмотрим архитектуру RNN для задачи языкового моделирования.

Шаг 1. Embedding

Есть входная последовательность из $n$ элементов. Мы пропускаем ее через слой эмбеддингов, который каждому элементу последовательности сопоставляет векторное представление.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/LSTM_lm_1.png" width="700"><center>
<center><em>Источник: <a href="https://www.researchgate.net/figure/The-recurrent-LSTM-language-model-structure-used-in-our-experiments_fig1_336086782">Investigation on N-gram Approximated RNNLMs for Recognition of Morphologically Rich Speech</a></em></center>

Шаг 2. LSTM1, LSTM2

На вход ячейки LSTM поступает вектор заданной длины. Скрытое состояние (краткосрочная память, hidden state) элемента $w_i$ передается на следующую ячейку того же слоя вместе с элементом $w_{i+1}$. На рисунке представлено 2 слоя LSTM. Следовательно, скрытое состояние элемента $w_i$ на слое LSTM1 передается также на следующий слой LSTM2.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/LSTM_lm_2.png" width="700"><center>
<center><em>Источник: <a href="https://www.researchgate.net/figure/The-recurrent-LSTM-language-model-structure-used-in-our-experiments_fig1_336086782">Investigation on N-gram Approximated RNNLMs for Recognition of Morphologically Rich Speech</a></em></center>

Шаг 3. Softmax

Выход каждого слоя — контекстуализированный эмбеддинг элемента последовательности. Однако в задаче языкового моделирования мы должны получать распределение  вероятностей для следующего элемента при условии текущей последовательности. Это должен быть вектор вероятностей, длина которого равна количеству элементов в словаре. Необходимо пропустить векторы элементов через линейный слой и применить функцию активации softmax.

Для элемента $w_i$ предсказывается вероятность элемента $w_{i+1}$ с учетом предшествующей последовательности $w_1, \cdots, w_i$ (по прошлому предсказываем будущее).

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/LSTM_lm_3.png" width="700"><center>
<center><em>Источник: <a href="https://www.researchgate.net/figure/The-recurrent-LSTM-language-model-structure-used-in-our-experiments_fig1_336086782">Investigation on N-gram Approximated RNNLMs for Recognition of Morphologically Rich Speech</a></em></center>

После предсказания вероятностей необходимо оценить качество и посчитать размер ошибки. В качестве правильных ответов мы используем те же обучающие данные, но сдвинутые на 1 шаг. Это достигается за счет добавления тегов начала START и конца END последовательности.

Для элемента START правильным ответом будет элемент h, для h — e, для e — l, и т.д.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/rnn_generation.png" width="600"></center>

<center><em>Источник: <a href="https://www.researchgate.net/figure/The-recurrent-LSTM-language-model-structure-used-in-our-experiments_fig1_336086782">Building Char-RNN</a></em></center>

Особенность обучающих данных для языкового моделирования состоит в том, что она не требует наличия размеченных данных: нужны только сырые тексты, на них модель обучается предсказывать продолжение последовательности.

Как только у нас будет языковая модель, мы сможем использовать ее для генерации текста. Мы делаем это по одному токену за раз: предсказываем распределение вероятности следующего токена с учетом предыдущего контекста и делаем выборку (sampling) из этого распределения.

<div align="center">
    <table >
     <tr>
       <td>
       
<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/generation_1.png" width="400"></center>
<em>1</em>
</td>

<td>
<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/generation_2.png" width="400"></center>
<em>2</em>
</td>

<td>
<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/generation_3.png" width="400"></center>
<em>3</em>
</td>
     </tr>
     
<div align="center">
    <table >
  <tr>
     <td>
<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/generation_4.png" width="400"></center>
<em>4</em>
</td>

<td>
<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/generation_5.png" width="400"></center>
<em>5</em>
</td>

<td>
<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/generation_6.png" width="400"></center>
<em>6</em>
</td>
     </tr>
    </table >

<center><em>Источник: <a href="https://lena-voita.github.io/nlp_course/language_modeling.html#n_gram_generation">NLP Course For You</a></em></center>

В качестве альтернативы можно применить жадный поиск (greedy decoding): на каждом шаге выбирается токен с наибольшей вероятностью. Однако обычно это работает не очень хорошо.

## Посимвольная генерация с помощью LSTM

Мы обучим языковую модель на уровне символов.

В качестве обучающих данных будем использовать [[doc] 🛠️ корпус названий динозавров](https://www.kaggle.com/datasets/swimmingwhale/dinosaur-island).

После обучения эта модель будет способна порождать новые имена динозавров, похожие на существующие.

### Загрузка и подготовка данных

Загрузим данных и посмотрим на содержание файла.

In [ ]:
!wget -q https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/datasets/dinos.txt

In [ ]:
print("First 10 dinos names:\n")
!head dinos.txt

In [ ]:
print("Last 10 dinos names:\n")
!tail dinos.txt

Определим доступную среду выполнения.

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

Создадим класс `DinoDataset`, наследник класса `Dataset`.

In [ ]:
from torch.utils.data import Dataset, DataLoader

class DinosDataset(Dataset):
  def __init__(self):
    super().__init__()
    with open('dinos.txt') as f:
      content = f.read().lower()
      self.vocab = sorted(set(content)) + ['<','>'] # добавляем в словарь все буквы, а также спецсимволы начала и конца
      self.vocab_size = len(self.vocab) # определяем размер словаря
      self.lines = content.splitlines() # разбиваем по строкам
    self.char2id = {char:id for id,char in enumerate(self.vocab)} # создаем словарь, каждому символу присваиваем индекс
    self.id2char = {id:char for id,char in enumerate(self.vocab)}

  def __getitem__(self, index):
    line = self.lines[index]
    """
    Input data x_str: special symbol for beginning of the sequence + sequence
    Output data y_str: sequence + special symbol for end of the sequence
    """
    x_str = '<' + line
    y_str = line + '>'
    x = torch.empty(len(x_str), dtype=torch.long, device=device) # создаем пустой тензор для входных данных
    y = torch.empty(len(y_str), dtype=torch.long, device=device) # создаем пустой тензор для выходных данных
    for i, (x_ch, y_ch) in enumerate(zip(x_str, y_str)): # переводим символы в индексы по словарю char2id
      x[i] = self.char2id[x_ch]
      y[i] = self.char2id[y_ch]
    return x,y

  def __len__(self):
    return len(self.lines) # определяем размер датасета

In [ ]:
dinos_dataset = DinosDataset()
dinos_dataloader = DataLoader(dinos_dataset, shuffle=True)

Убедимся, что входные и выходные данные различаются сдвигом на один шаг.

In [ ]:
x,y = next(iter(dinos_dataloader))
print(f"Shape of input sequence: {x.shape}")
print(f"Input sequence (indexes):\n{x}")
print(f"Input sequence (symbols):\n{[dinos_dataset.id2char[int(i)] for i in x[0]]}\n")
print(f"Shape of output sequence: {y.shape}")
print(f"Output sequence (indexes):\n{y}")
print(f"Output sequence (symbols):\n{[dinos_dataset.id2char[int(i)] for i in y[0]]}")

Посмотрим на размер обучающих данных и количество уникальных символов.

In [ ]:
print(f"Number of unique symbols: {len(dinos_dataset.lines)}")
print(f"Length of dataset: {dinos_dataset.vocab_size}")

### Создание модели

Перейдем к построению модели. Она включает эмбеддинг слой, два слоя LSTM и линейный слой.

In [ ]:
from torch import nn

class LM(nn.Module):
  def __init__(self, vocab_size):
    super(LM, self).__init__()
    self.lstm_size = 15 # размер скрытых состояний h и c (краткосрочная и долгая память)
    self.embedding_dim = 10 # размер входных данных (длина эмбеддингов)
    self.num_layers = 2 # количество слоев LSTM

    # слой эмбеддингов
    self.embedding = nn.Embedding(
        num_embeddings=vocab_size,
        embedding_dim=self.embedding_dim
        )
    # слой LSTM
    self.lstm = nn.LSTM(
        input_size=self.embedding_dim,
        hidden_size=self.lstm_size,
        num_layers=self.num_layers
    )
    # линейный слой
    self.hid2out = nn.Linear(
        in_features=self.lstm_size,
        out_features=vocab_size
        )

  def forward(self, x, prev_state=None):
    embedding = self.embedding(x)
    if prev_state:
      output, state = self.lstm(embedding)
    else:
      output, state = self.lstm(embedding, prev_state)
    logits = self.hid2out(output)

    return logits, state

In [ ]:
model = LM(len(dinos_dataset.char2id)).to(device)

Применим модель к одному батчу и посмотрим размер получившихся данных.

In [ ]:
y_pred, (state_h, state_c) = model(x)
print(f"Shape of prediction: {y_pred.shape}")
print(f"Shape of hidden state: {state_h.shape}")
print(f"Shape of cell state: {state_c.shape}")

### Случайная выборка из элементов массива

Метод `np.random.choice` выбирает из списка элемент случайным образом. Если мы не задаем распределение вероятностей, то каждый элемент будет выбран примерно одинаковое количество раз.

In [ ]:
import numpy as np

print("Sampling in range from 1 to 3:")
for i in range(10):
  print(np.random.choice([1,2,3]))

Распределение вероятностей может быть задано эксплицитно. Если это ohe-hot вектор, то всегда будет выбираться номер позиции, на которой стоит 1.

In [ ]:
p1 = torch.nn.functional.one_hot(torch.tensor(2), len(dinos_dataset.char2id))
print(f"One-hot vector p1:\n{p1}")

In [ ]:
print(f"Sampling with one-hot vector p1:")

for i in range(10):
  pred_id = np.random.choice(np.arange(len(dinos_dataset.char2id)), p=p1)
  print(pred_id)

Можно использовать распределение вероятностей, которое было получено при обучении сети. Для этого необходимо к выходным данным применить функцию активации softmax.

In [ ]:
logits = y_pred[:, -1, :].unsqueeze(1)
print(f"Shape of logits: {logits.shape}")
print(f"Logits:\n{logits}\n")
y_softmax_scores = torch.softmax(logits, dim=2)
print(f"Shape of predictions after softmax: {y_softmax_scores.shape}")
print(f"Predictions after softmax:\n{y_softmax_scores}")
print(f"Sum of presictions: {torch.sum(y_softmax_scores)}")

In [ ]:
p2 = y_softmax_scores.detach().cpu().numpy().ravel()
print(f"Numpy array of predictions after softmax p2:\n{p2}")

In [ ]:
print(f"Sampling with predictions after softmax p2:")
for i in range(20):
  pred_id = np.random.choice(np.arange(len(dinos_dataset.char2id)), p=p2)
  print(pred_id)

### Генерация

Напишем функцию для генерации названий динозавров.

In [ ]:
def inference(dataset, model):
  model.eval() # переводим модель в состояние тестирования
  newline_id = dataset.char2id['>'] # записываем индекс символа конца последовательности
  word_size = 0 # будем контролировать длину порождаемой последовательности
  with torch.no_grad():
    state_h, state_c = (None, None) # скрытые состояния будут передаваться вручную, поэтому их надо хранить
    start_id = dataset.char2id['<'] # генерация начинается с символа начала последовательности
    indices = [start_id] # создаем список, где будем хранить предсказания
    word_size += 1 # увеличиваем длину последовательности
    pred_id = start_id # записываем символ начала последовательности как первое предсказание
    x = torch.tensor([[pred_id]]).to(device) # преобразуем в тензор

    """
    Будем использовать два условия для продолжения генерации в цикле while:
    1) сгенерированный символ не является символом конца последовательности '>'
    и
    2) длина сгенерированной последовательности меньше 20
    """
    while pred_id != newline_id and word_size < 20:
      logits, (state_h, state_c) = model(x, (state_h, state_c)) # передаем в модель тензор с текущим символом x, предыдущие состояния h и c
      y_softmax_scores = torch.softmax(logits, dim=2) # применяем softmax к предсказаниям модели
      pred_id = np.random.choice( # осуществляем случайную выборку значений из заданного массива
          np.arange(len(dinos_dataset.char2id)), # с некоторой вероятностью получим один из 29 индексов
          p=y_softmax_scores.detach().cpu().numpy().ravel() # вероятность определяется в зависимости от обучающих данных
          )
      indices.append(pred_id) # добавляем предсказанный индекс в список предсказаний
      x = torch.tensor([[pred_id]]).to(device) # преобразуем в тензор
      word_size += 1 # увеличиваем длину последовательности

      if word_size == 20 and indices[-1] != newline_id:
        indices.append(newline_id)

    model.train()

    return ''.join([dinos_dataset.id2char[i] for i in indices]) # возвращаем индексы, переведенные в символы

### Обучение модели

Напишем функцию для обучения модели.

In [ ]:
def train(dataset, dataloader, model, criterion, optimizer, max_epochs):
  model.train() # переводим модель в состояние обучения
  losses = []
  for epoch in range(max_epochs):
    print(f'\nEpoch {epoch+1}')
    epoch_loss = 0
    for batch, (x,y) in enumerate(dataloader):
      optimizer.zero_grad()
      y_pred, (state_h, state_c) = model(x) # передаем данные в модель, записываем предсказание
      loss = criterion(y_pred.transpose(1,2), y) # считаем ошибку
      epoch_loss += loss.item()
      loss.backward()
      optimizer.step()
      if (batch+1) % 100 == 0:
        print(inference(dataset, model))

    print(f'Loss {epoch_loss/(batch+1)}')
    losses.append(epoch_loss/(batch+1))
  return losses

Создадим модель, определим функцию потерь, метод оптимизации и количество эпох.

In [ ]:
from torch import optim
model = LM(len(dinos_dataset.char2id)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.2)
max_epochs = 15

Запустим обучение. Будем записывать значения ошибки и выводить сгенерированные последовательности.

In [ ]:
losses = train(dinos_dataset,dinos_dataloader, model, criterion, optimizer, max_epochs)

Выведем значения функции потерь на графике.

In [ ]:
import matplotlib.pyplot as plt
plt.plot(losses)
plt.title('Cross Entropy Loss value')
plt.ylabel('Cross Entropy Loss')
plt.xlabel('epoch')
plt.show()

Для оценки языковых моделей используется перплексия. Она показывает, насколько хорошо модель аппроксимирует истинное распределение вероятностей, лежащее в основе данных. Чем меньше перплексия, тем лучше модель.

[[blog] ✏️ Что такое перплексия?](https://training.continuumlabs.ai/data/datasets/what-is-perplexity)

# RNN для машинного перевода

Другая важная задача обработки естественного языка — машинный перевод: последовательности на одном языке сопоставляется последовательность на другом языке, передающая тот же смысл.

Рассмотрим задачу перевода с английского на французский язык:
- Исходное предложение: I am a student.
- Целевое предложение: Je suis étudiant.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/machine_translation.png" width="600"></center>

Что нужно для перевода текста?
1. Понимать отдельные слова
2. Понимать взаимодействие между словами (синтаксис)
3. Переводить слова (с учетом контекста)
4. Составлять осмысленный и связный текст

Для перевода текста можно разбить исходное предложение на несколько фрагментов, затем переводить его по фразам.

Однако у такого подхода есть проблемы:
- одному слову в исходном предложении (*forced*) может соответствовать несколько слов в целевом предложении (*a forcé*)
- порядок слов может меняться (*exceptional measures* vs. *des mesures exceptionnelles*)

Следовательно, возникают сложности с построением выравнивания слов (word alignment).

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L04/out/word_alignment.png" width="800"></center>

Другая возможность — прочитать все исходное предложение целиком, проанализировать его значение, а затем произвести перевод.

В этом случае модель состоит из двух блоков:
- кодировщик: строит векторное представление для исходного предложения (кодирует);
- декодировщик: генерирует целевое предложение на основе этого векторного представления (раскодирует).

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/encoder_decoder.png" width="700"></center>

<center><em>Источник: <a href="https://blog.paperspace.com/introduction-to-neural-machine-translation-with-bahdanaus-attention/">Introduction to Neural Machine Translation</a></em></center>

Машинный перевод относится к задаче преобразования одной последовательности в другую, длина которых может быть любой и не обязательно должна совпадать: "многие ко многим" или sequence-to-sequence (сокращенно seq2seq).

## Архитектура кодировщик-декодировщик

Модель для машинного перевода состоит из блока <font color="#5b9b2c">кодировщика</font> и <font color="#9b2c6a">декодировщика</font>.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/enc_dec-min.png" width="500"></center>

<center><em>Источник: <a href="https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html">NLP Course For You</a></em></center>

### Кодировщик

Архитектура <font color="#5b9b2c"> кодировщика </font>:

- на вход первой ячейки RNN поступает нулевое скрытое состояние;
- его вектор обрабатывается внутри первой ячейки кодировщика;

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/seq2seq_1.png" width="700"></center>

<center><em>Источник: <a href="https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html">NLP Course For You</a></em></center>

- на вход второй ячейки RNN поступает вектор слова "Я" и нулевое скрытое состояние;
- вектор слова "Я" обрабатывается внутри второй ячейки кодировщика;

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/seq2seq_2.png" width="700"></center>

<center><em>Источник: <a href="https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html">NLP Course For You</a></em></center>

- на вход третьей ячейки поступает вектор слова "видел" и измененный вектор слова "Я" как скрытое состояние;
- вектор слова "видел" обрабатывается внутри третьей ячейки кодировщика;

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/seq2seq_3.png" width="700"></center>

<center><em>Источник: <a href="https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html">NLP Course For You</a></em></center>

- то же самое происходит со всеми словами исходного предложения;

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/seq2seq_4.png" width="700"></center>

<center><em>Источник: <a href="https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html">NLP Course For You</a></em></center>

- вектор спецтокена \<eos\>, обозначающего конец предложения, обрабатывается внутри последней ячейки кодировщика;
- измененный вектор спецтокена \<eos\> (= вектор предложения) поступает на вход декодировщика как скрытое состояние.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/seq2seq_5.png" width="700"></center>

<center><em>Источник: <a href="https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html">NLP Course For You</a></em></center>

### Декодировщик

Архитектура <font color="#9b2c6a"> декодировщика</font>:
- на вход первой ячейки декодировщика поступает нулевое скрытое состояние и вектор исходного предложения из кодировщика как скрытое состояние;
- они обрабатываются внутри первой ячейки декодировщика;

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/seq2seq_6.png" width="700"></center>

<center><em>Источник: <a href="https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html">NLP Course For You</a></em></center>

- измененный вектор нулевого скрытого состояния передается на линейный слой, применяется softmax;
- на выходе получаем вектор, длина которого равна длине словаря, — это распределение вероятностей для следующего элемента при условии текущей последовательности;

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/seq2seq_7.png" width="700"></center>

<center><em>Источник: <a href="https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html">NLP Course For You</a></em></center>

- сравниваем полученное распределение вероятностей с правильным ответом;

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/seq2seq_8.png" width="700"></center>

<center><em>Источник: <a href="https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html">NLP Course For You</a></em></center>

- подсчитываем значение функции потерь и обновляем веса так, чтобы вероятность правильного слова была выше, а вероятности остальных слов — ниже;

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/seq2seq_9.png" width="700"></center>

<center><em>Источник: <a href="https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html">NLP Course For You</a></em></center>

- записываем предсказание и переходим к следующему токену;
- на вход второй ячейки поступает вектор слова "I" и измененный вектор нулевого скрытого состояния;
- вектор слова "I" обрабатывается внутри второй ячейки декодировщика;

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/seq2seq_10.png" width="700"></center>

<center><em>Источник: <a href="https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html">NLP Course For You</a></em></center>

- измененный вектор слова "I" передается на линейный слой, применяется softmax, на выходе получаем распределение вероятностей;

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/seq2seq_11.png" width="700"></center>

<center><em>Источник: <a href="https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html">NLP Course For You</a></em></center>

- сравниваем полученное распределение с правильным ответом, подсчитываем значение функции потерь и обновляем веса;

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/seq2seq_12.png" width="700"></center>

<center><em>Источник: <a href="https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html">NLP Course For You</a></em></center>

- то же самое происходит со всеми словами;
- на выходе получаем 8 векторов, соответствующих распределению вероятностей для каждого слова.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/seq2seq_13.png" width="700"></center>

<center><em>Источник: <a href="https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html">NLP Course For You</a></em></center>

## Недостатки стандартной модели

Итак,
- <font color="#5b9b2c">кодировщик</font> используется для получения конекстуализированного вектора исходного предложения;
-  <font color="#9b2c6a">декодировщик</font> используется для генерации целевого предложения;
- вектор исходного предложения используется в качестве скрытого состояния на первом шаге генерации.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/seq2seq_14.png" width="700"></center>

<center><em>Источник: <a href="https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html">NLP Course For You</a></em></center>

**Проблема Sequence-to-Sequence**: использование только конечного скрытого состояния <font color="#5b9b2c">кодировщика</font> для представления всей входной последовательности приводит к потере информации, особенно с начала последовательности;
- по мере того, как <font color="#5b9b2c">кодировщик</font> обрабатывает входную последовательность, ожидается, что в конечном скрытом состоянии будет собрана вся необходимая информация;
- когда последовательность становится длиннее, этому единственному состоянию труднее сохранять всю необходимую информацию из предыдущих частей последовательности.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/bottleneck.png" width="500"></center>

<center><em>Источник: <a href="https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html">NLP Course For You</a></em></center>

## Механизм внимания

На разных этапах декодировщику может потребоваться сосредоточиться на разных исходных лексемах, которые более актуальны на данном этапе. Давайте посмотрим на весовые коэффициенты внимания: они показывают, какие исходные слова оказываются "важнее" для декодировщика при генерации целевой последовательности.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/attention_matrices.jpg" width="800"></center>

<center><em>Источник: <a href="https://arxiv.org/abs/1409.0473">Neural Machine Translation by Jointly Learning to Align and Translate</a></em></center>

Чтобы модель могла фокусироваться на различных частях входной последовательности и сохранять больше информации на протяжении всего процесса кодирования и декодирования, был предложен механизм внимания (attention mechanism).

[[paper] 🎓 Bahdanau D., Cho K., Bengio Y. (2014). Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473)

На каждом этапе <font color="#9b2c6a">декодирования</font> механизм внимания решает, какие части исходного предложения являются более важными. В этом случае <font color="#5b9b2c">кодировщику</font> не нужно сжимать все исходное предложение в один вектор — он выдает векторные представления для всех токенов, все состояния RNN.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/attention_1.png" width="700"></center>

<center><em>Источник: <a href="https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html">NLP Course For You</a></em></center>

Архитектура <font color="#5b9b2c">кодировщика</font> не меняется, все изменения касаются архитектуры <font color="#9b2c6a">декодировщика</font>.

Рассмотрим изменения пошагово на примере обработки слова "а":

- на вход ячейки декодировщика поступает вектор слова "a" и измененный вектор слова "saw" как скрытое состояние;
- вектор слова "a" обрабатывается внутри ячейки декодировщика;
- выход ячейки декодеровщика, соответствующей слову "a", сравнивается с выходом первой ячейки кодировщика, соответствующей слову "Я", с помощью некоторой функции сходства;

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/attention_2.png" width="450"></center>

<center><em>Источник: <a href="https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html">NLP Course For You</a></em></center>

- аналогичная мера сходства считается для вектора слова "a" и векторов каждого слова исходной последовательности;

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/attention_3.png" width="450"></center>

<center><em>Источник: <a href="https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html">NLP Course For You</a></em></center>

- чтобы интерпретировать полученные значения как веса, применяется функция softmax — теперь их сумма равна 1;
- полученные значения называются весами внимания (attention weights): если векторы некоторого слова целевого и исходного предложения схожи, то вес внимания будет большим, и при переводе (генерации) данного слова декодировщик больше "обращает внимания" на него;

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/attention_4.png" width="600"></center>

<center><em>Источник: <a href="https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html">NLP Course For You</a></em></center>

- вектор каждого слова исходного предложения умножается на свой вес внимания, затем все векторы складываются, получаем контекстный вектор (attention output);

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/attention_5.png" width="600"></center>

<center><em>Источник: <a href="https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html">NLP Course For You</a></em></center>

- вектор слова "a" и контекстный вектор конкатенируются;
- конкатенированный вектор передается на линейный слой, применяется softmax;
- получаем вектор, длина которого равна длине словаря, — это распределение вероятностей для следующего элемента при условии спецсимвола начала предложения;

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/attention_6.png" width="650"></center>

<center><em>Источник: <a href="https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html">NLP Course For You</a></em></center>

- то же самое происходит со всеми словами;
- на выходе получаем 8 векторов, соответствующих распределению вероятностей для каждого слова.

<center><img src="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L04/attention_7.png" width="750"></center>

<center><em>Источник: <a href="https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html">NLP Course For You</a></em></center>